In [3]:
# import miscellaneous modules
import matplotlib.pyplot as plt
import os
import numpy as np
import time
import keras
import tensorflow as tf
import tensorflow_datasets as tfds
keras.__version__
tf.__version__

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPU


In [12]:
(train_ds, validation, test_ds), info = tfds.load("beans", split=['train', 'validation', 'test'],
                                      as_supervised=True, 
                                      with_info=True)

In [13]:
info

tfds.core.DatasetInfo(
    name='beans',
    full_name='beans/0.1.0',
    description="""
    Beans is a dataset of images of beans taken in the field using smartphone
    cameras. It consists of 3 classes: 2 disease classes and the healthy class.
    Diseases depicted include Angular Leaf Spot and Bean Rust. Data was annotated by
    experts from the National Crops Resources Research Institute (NaCRRI) in Uganda
    and collected by the Makerere AI research lab.
    """,
    homepage='https://github.com/AI-Lab-Makerere/ibean/',
    data_dir='/home/ubuntu/tensorflow_datasets/beans/0.1.0',
    file_format=tfrecord,
    download_size=171.69 MiB,
    dataset_size=171.63 MiB,
    features=FeaturesDict({
        'image': Image(shape=(500, 500, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=3),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=128, num_shards=1>,
        'train':

In [22]:
train_ds

<_PrefetchDataset element_spec=(TensorSpec(shape=(500, 500, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [16]:
len(test_ds)

128

In [15]:
len(train_ds)

1034

**Rescaling Layer**

เลเยอร์นี้เป็นเลเยอร์ที่นำมาทดแทนการทำ normalisation 
เมื่อก่อนเราอาจจะต้องทำการหาร numpy array ด้วย 255. เพื่อปรับให้ช่วงข้อมูลสีของรูปภาพนั้นอยู่ในช่วงระหว่าง [0., 1.]

อย่างไรก็ตาม ในเมื่อกระบวนการนี้นั้นต้องทำกับรูปภาพ training และ testing อยู่แล้ว ผู้พัฒนาบางก็เลยถือโอกาศใส่ให้มันเป็นส่วนหนึ่งของโมเดลไปเลย
แต่มีข้อเสียนิดหน่อยตรงที่ไม่สามารเรียกข้อมูลหลัง normalistaion ออกมาดูได้นั่นเอง

In [26]:
rescale = tf.keras.Sequential([
    tf.keras.layers.Resizing(200,200),
  tf.keras.layers.Rescaling(1./255)
])

**Augmentation เลเยอร์อื่นที่ keras.layers นั้นมีให้ มีตัวไหนอีก??**

อยากให้ลองเรียก เลเยอร์อื่นๆมา่ใส่เพิ่มอีกสักตัวสองตัว ส่วนใหญ่จะเป็นตระกูล tf.keras.layers.Random...... ทั้งหลาย

In [27]:
augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomZoom([0.01,0.05])
])

In [29]:
model = tf.keras.Sequential([
    # Add the preprocessing layers you created earlier.
    tf.keras.Input(shape=(500,500,3)),
    rescale,
    augmentation,
    tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu', input_shape=(200,200,3)),
    tf.keras.layers.MaxPooling2D(),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(10,activation="softmax")
])

/home/ubuntu/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [31]:
model.compile(optimizer=keras.optimizers.Adam(), 
              loss="sparse_categorical_crossentropy",
              metrics=['accuracy']
             )
model.summary()

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_7 (Sequential)       │ (None, 200, 200, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_8 (Sequential)       │ (None, 200, 200, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 200, 200, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 100, 100, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 160000)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │    10,240,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,241,162 (39.07 MB)

 Trainable params: 10,241,162 (39.07 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(train_ds.batch(32), epochs=5)

In [34]:
model.predict(test_ds.batch(32))

W0000 00:00:1727928499.027137   31743 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727928499.052153   31743 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727928499.067328   31743 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727928499.085240   31743 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727928499.103277   31743 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727928499.119698   31743 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727928499.147368   31743 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727928499.166342   31743 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727928499.185008   31743 gp

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step  


W0000 00:00:1727928499.445344   31743 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727928499.453033   31743 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727928499.465951   31743 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727928499.477473   31743 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727928499.488420   31743 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727928499.498908   31743 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced


array([[3.4584972e-01, 6.4005208e-01, 1.4098231e-02, ..., 1.3072320e-14,
        4.6764654e-12, 7.0347444e-13],
       [4.3144059e-01, 5.4153877e-01, 2.7014725e-02, ..., 3.6957513e-08,
        9.4843848e-08, 1.8515941e-07],
       [1.7052446e-03, 1.1704600e-02, 9.8659015e-01, ..., 1.5570122e-13,
        5.0090834e-12, 1.6621615e-12],
       ...,
       [2.6477579e-04, 8.1241708e-03, 9.9161106e-01, ..., 3.0345225e-12,
        9.0650668e-11, 1.5203465e-11],
       [3.3476582e-01, 5.1977271e-01, 1.4546147e-01, ..., 6.3098899e-17,
        2.0146890e-14, 1.9704140e-15],
       [3.5313562e-02, 9.2401820e-01, 4.0664386e-02, ..., 1.7107683e-07,
        1.8271487e-07, 8.7235534e-08]], dtype=float32)